## Part 1: LSTM for Sequence Classification on the IMDB dataset

In a first exercise we want to build a model that can classify movie reviews as positive or negative based on their sentiment. To do so we train a model using the IMDB dataset.

The IMDB dataset consists of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

In [1]:
# # Install packages:
# pip install torchdata
# pip install torchtext

# Additionally you may need:
# pip install portalocker

In [2]:
import numpy as np
import numpy
import pandas as pd
# fix random seed so every student get the same results
numpy.random.seed(7)

import torch
from torch import nn
from torch.nn import functional as F
from torchtext.datasets import imdb
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

%matplotlib inline
from matplotlib import pyplot as plt

We need to load the IMDB dataset. We are constraining the dataset to the top 5,000 words. We also split the dataset into train (50%) and test (50%) sets.

In [3]:
class VocabOneHot(): # Helper class for processing the
    def __init__(self,
                 tokenizer,
                 num_words = 5000):
        self.counts = {} # Dictionary for storing the counts of each word
        self.tokenizer = tokenizer
        self.num_words = num_words # maximal number of words to use
        self.fitted = False
        self.lookup = {} # lookup table assigning to each word an integer

    def fit(self, x_iter):
        self.counts = {} # Reinitialize the counts
        for label, line in train_iter: #iterate over the instances
            tokens = self.tokenizer(line)
            for t in tokens:
                if t in self.counts:
                    self.counts[t] += 1
                else:
                    self.counts[t] = 1
        count_series = pd.Series(self.counts).sort_values(ascending=False) # create Series for easy manipulation
        self.lookup = {word[0]:i+1 if i<self.num_words-2 else self.num_words-1
                       for i, word in enumerate(count_series.reset_index().to_numpy())}
        self.fitted = True

    def transform(self, tokenized_sentence, length = 100):
        assert self.fitted, "The instance has not been fitted yet"
        non_padded= [self.lookup[word]
                if word in self.lookup.keys()
                else self.num_words-1
                    for word in tokenized_sentence]
        non_padded = np.array(non_padded) # create unpadded version
        if len(non_padded) == length: # leave the sequence as it stands
            return non_padded
        elif len(non_padded) > length: # truncate
            return non_padded[:length]
        else: # pad
            padding = np.zeros(length - len(non_padded))
            padded = np.hstack([padding, non_padded]).astype(int)
            return padded

In [4]:
train_iter = imdb.IMDB("data/",  split="train")
test_iter = imdb.IMDB("data/", split="test")
tokenizer = get_tokenizer("basic_english", language='en')

In [5]:
# Could be necessary depending of the version of torchtext
train_iter = list(train_iter)
test_iter = list(test_iter)

In [6]:
vocab = VocabOneHot(tokenizer) # Initialize the vocabulary

In [7]:
sr = vocab.fit(train_iter) # Fit the vocabulary to the training set

In [8]:
X_train = []
y_train = []
ys = []
for i, (label, line) in enumerate(train_iter): # Create training data
    X_train += [vocab.transform(tokenizer(line))]
    y_train += [label]
X_train = np.array(X_train).astype(int)
y_train = np.array(y_train)
X_test = []
y_test = []
for i, (label, line) in enumerate(test_iter): # Create test data
    X_test += [vocab.transform(tokenizer(line))]
    y_test += [label]
X_test = np.array(X_test).astype(int)
y_test = np.array(y_test)
y_test[y_test==1] = 0
y_test[y_test==2] = 1
y_train[y_train==1] = 0
y_train[y_train==2] = 1

<b>Exercise 1</b>:

Create the training data by selecting balanced subsets of the training and test splits, each of size 5000.

In [9]:
n_pos_train_samples = np.where(y_train == 0)
n_neg_train_samples = np.where(y_train == 1)

n_pos_test_samples = np.where(y_test == 0)
n_neg_test_samples = np.where(y_train == 1)

positives_train = X_train[y_train == 0]
negatives_train = X_train[y_train == 1]
positives_test = X_test[y_test == 0]
negatives_test = X_test[y_test == 1]

In [10]:
X_train = np.concatenate((X_train[n_pos_train_samples[0][:2500]], X_train[n_neg_train_samples[0][:2500]]))
y_train = np.concatenate((y_train[n_pos_train_samples[0][:2500]], y_train[n_neg_train_samples[0][:2500]]))

X_test = np.concatenate((X_test[n_pos_train_samples[0][:2500]], X_test[n_neg_train_samples[0][:2500]]))
y_test = np.concatenate((y_test[n_pos_train_samples[0][:2500]], y_test[n_neg_train_samples[0][:2500]]))

In [11]:
try:
    assert X_train.shape == (5000, 100)
    assert X_test.shape == (5000, 100)
    print("Testing successful.")
except:
    print("Tests failed.")

Testing successful.


We can now define and fit our LSTM model.

The first layer is the embedded layer that uses 32 length vectors to represent
 each word. The next layer is the LSTM layer with 5 memory units (smart neurons).
 Finally, because this is a classification problem we use a dense output
 layer with a single neuron and a sigmoid activation function to make 0 or 1
 predictions for the two classes (good or bad) in the problem.

Because it is a binary classification problem, log loss is used as the loss
function. The efficient ADAM optimization
algorithm is used to do the gradient descent.

<b>Exercise 2</b>:

Explore the class LSTM_model, try to understand the details of the forward method, and comment the code

In [12]:
# Create dataloaders containing the test and training data
data_train = torch.utils.data.TensorDataset(torch.Tensor(X_train).long(),
                               torch.Tensor(y_train).long())
data_test = torch.utils.data.TensorDataset(torch.Tensor(X_test).long(),
                               torch.Tensor(y_test).long())
train_dl = torch.utils.data.DataLoader(data_train, batch_size = 256, shuffle=True)
test_dl = torch.utils.data.DataLoader(data_test, batch_size = 256)

In [13]:
class RecurrentDropout(nn.Module):
    """
    Module for Dropout in recurrent architectures.
    The mask is initialized on the first iteration, and used along the sequence.
    """
    def __init__(self, p = 0.1):
        super().__init__()
        self.id = nn.Identity()
        self.bern = torch.distributions.Bernoulli(p)
        self.p = p
    def forward(self, x, first=True):
        if self.training:
            if first:
                self.mask = self.bern.sample([x.shape[0], x.shape[1]])
            else:
                pass
            x = self.id(x)
            x[self.mask.bool()] = 0
            return (x / (1 - self.p))
        else:
            return self.id(x)

In [14]:
class LSTM_model(nn.Module):
    def __init__(self,
                num_embeddings = 5000,
                embedding_dim = 32,
                lstm_dim = 10,
                target_size = 1,
                recurrent_dropout = 0,
                seq_dropout = 0):
        super().__init__()
        self.lstm_dim = lstm_dim
        self.embedding = nn.Embedding(num_embeddings,
                                     embedding_dim)
        self.lstm = nn.LSTMCell(embedding_dim, lstm_dim)
        self.clf = nn.Linear(lstm_dim, target_size)
        self.recurrent_do = RecurrentDropout(p = recurrent_dropout)
        self.seq_dropout = nn.Dropout1d(p=seq_dropout)

    def forward(self, x, autoregressive_iterations=None):
        x = self.embedding(x)
        x = self.seq_dropout(x)
        for i in range(x.shape[-2]):
            if i == 0:
                hn, cn = self.lstm(x[:, i])
                hn = self.recurrent_do(hn, first=True)
            else:
                hn, cn = self.lstm(x[:, i], (hn,
                                             cn))
                hn = self.recurrent_do(hn, first=False)
        if autoregressive_iterations is None:
            return self.clf(hn)
        else:
            output = []
            for it in range(autoregressive_iterations):
                token = self.clf(hn).argmax(axis=1)
                output += token
                embed_token = self.embedding(token)
                hn, cn = self.lstm(embed_token, (hn, cn))
        return torch.stack(output)


Now the boilerplate code used for training and evaluating the model will be defined

In [15]:
def train_batch(model, optimizer, loss_fn, dataloader):
    losses = []
    hit = 0
    miss = 0
    model.train()
    for batch in dataloader:
        batch[0] = batch[0].to(device)
        batch[1] = batch[1].to(device)
        output = model(batch[0])
        loss = loss_fn(output.squeeze(), batch[1].squeeze().float())
        losses += [loss.item()]
        loss.backward()
        pred =  (output > 0).long()
        equal = pred.squeeze() == batch[1].squeeze()
        hit += equal.sum()
        miss += (~equal).sum()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        optimizer.zero_grad()
    return np.mean(losses), hit/(hit + miss)

def test_batch(model, optimizer, loss_fn, dataloader):
    losses = []
    hit = 0
    miss = 0
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch[0] = batch[0].to(device)
            batch[1] = batch[1].to(device)
            output = model(batch[0])
            loss = loss_fn(output.squeeze(), batch[1].squeeze().float())
            losses += [loss.item()]
            pred =  (output > 0).long()
            equal = pred.squeeze() == batch[1].squeeze()
            hit += equal.sum()
            miss += (~equal).sum()
    return np.mean(losses), hit/(hit + miss)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTM_model()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), 0.001)
loss_fn = nn.BCEWithLogitsLoss()

<b>Exercise 3:</b>

Train a model for only 10 epochs and evaluate the model on the test data. Use a batch size of 64 reviews to update the weights in the LSTM. Use X_test and y_test to validate the model.

In [17]:
for epoch in range(10):
    train_loss =  train_batch(model, optimizer, loss_fn, train_dl)
    test_loss = test_batch(model, optimizer, loss_fn, test_dl)
    print(f"""Epoch: {epoch},
    train_loss: {train_loss[0]},
    train_acc: {train_loss[1]},
    test_loss: {test_loss[0]},
    test_acc: {test_loss[1]}""")

Epoch: 0,
    train_loss: 0.699948838353157,
    train_acc: 0.501800000667572,
    test_loss: 0.6994002461433411,
    test_acc: 0.501800000667572
Epoch: 1,
    train_loss: 0.6946202576160431,
    train_acc: 0.5062000155448914,
    test_loss: 0.6954572677612305,
    test_acc: 0.5081999897956848
Epoch: 2,
    train_loss: 0.6909187018871308,
    train_acc: 0.522599995136261,
    test_loss: 0.6930169522762298,
    test_acc: 0.5212000012397766
Epoch: 3,
    train_loss: 0.6883231550455093,
    train_acc: 0.545799970626831,
    test_loss: 0.691794353723526,
    test_acc: 0.5249999761581421
Epoch: 4,
    train_loss: 0.6860994905233383,
    train_acc: 0.5583999752998352,
    test_loss: 0.6910153925418854,
    test_acc: 0.5242000222206116
Epoch: 5,
    train_loss: 0.6831290155649186,
    train_acc: 0.5691999793052673,
    test_loss: 0.6905044257640839,
    test_acc: 0.5271999835968018
Epoch: 6,
    train_loss: 0.6799813061952591,
    train_acc: 0.5812000036239624,
    test_loss: 0.68986327350139

<b>Exercise 4:</b>

Once we fitted the model, we estimate the performance of the model on unseen reviews.


Modify the test_batch function to create predictions for the test set (note that the model outputs logits and no probabilities, hint: torch.sigmoid).

Predict the instances found in the test dataloader, use a ROC curve to show the performance of the model.

What conclusions can you draw from the ROC curve?

In [18]:
def predict(model, optimizer, loss_fn, dataloader):
   # set model to evaluate model
    model.eval()
    
    y_true = torch.tensor([], dtype=torch.int)
    all_outputs = torch.tensor([])
    
    with torch.no_grad():
        for batch in dataloader:
            outputs = model(batch[0])
            y_true = torch.cat((y_true, batch[1]), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)
    
    return torch.sigmoid(all_outputs), y_true


In [19]:
y_preds, y_true = predict(model, optimizer, loss_fn, test_dl)

## Part 2: Generate sentences using LSTMs

In this part of the lab we are using a pretrained model (you can train the model on your own, but this will take some time). This model was trained to predict the next character given a sequence of 100 previous characters. This model can be used to generate new sentences/phrases. Your task is to use this model and generate sentences with it. The model was pretrained on a book called "wonderland".

First of all we want to load the data:

In [21]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename,encoding="utf8").read()
raw_text = raw_text.lower()
# if there is an error use (python 2 vs. python 3):
#raw_text = open(filename).read()
#raw_text = raw_text.lower().decode('utf-8')

Now that the book is loaded, we must prepare the data for modeling by the neural network. We cannot model the characters directly, instead we must convert the characters to integers.

We can do this easily by first creating a set of all of the distinct characters in the book, then creating a map of each character to a unique integer.

In [22]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

For example, the list of unique unicode sorted lowercase characters in the book is as follows:

In [23]:
print(chars)

['\n', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']


You can see that there may be some characters that we could remove to further clean up the dataset that will reduce the vocabulary and may improve the modeling process. In this lab we skip this process.

Now that the book has been loaded and the mapping prepared, we can summarize the dataset.

In [24]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: " + str(n_chars))
print("Total Vocab: " + str(n_vocab))

Total Characters: 163817
Total Vocab: 61


We can see that the book has just over 160,000 characters and that when
converted to lowercase that there are only 61 distinct characters in the vocabulary for the network to learn.

We now need to define the training data for the network. There is a lot of flexibility in how you choose to break up the text and expose it to the network during training.

In this tutorial (as explained above) we will split the book text up into subsequences with a fixed length of 100 characters.

Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y). When creating these sequences, we slide this window along the whole book one character at a time, allowing each character a chance to be learned from the 100 characters that preceded it.

For example, if the sequence length is 5 (for simplicity) then the first two training patterns would be as follows:

CHAPT -> E, HAPTE -> R

As we split up the book into these sequences, we convert the characters to integers using our lookup table we prepared earlier.

<b>Exercise 5:</b>

Create all patterns.

Create a list of sequences dataX that contains all windows of the book and a list of following characters dataY.
So each entry in the list dataX contains of a vector of 100 integer values representing the characters that occur in the window.
Each entry in the list dataY contains the following character for the associated window in dataX.

Hint: Use the dictionary char_to_int to map the characters to integers. You can find an example entry of dataX and dataY in the test cell below.

In [41]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i, _ in enumerate(raw_text):
    if i + seq_length >= len(raw_text):
        break
    dataX.append([char_to_int[character] for character in raw_text[i: i + seq_length]])
    dataY.append(char_to_int[raw_text[i + seq_length]])

n_patterns = len(dataX)
print("Total Patterns: " +  str(n_patterns))

Total Patterns: 163717


Check if everything is correct:

In [42]:
try:
    #assert n_patterns >= 163717
    assert dataX[0] == [60, 45, 47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 30, 41, 38, 32, 34, 57, 48, 1, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 1, 38, 43, 1, 52, 44, 43, 33, 34, 47, 41, 30, 43, 33, 9, 1, 31, 54, 1, 41, 34, 52, 38, 48, 1, 32, 30, 47, 47, 44, 41, 41, 0, 0, 49, 37, 38, 48, 1, 34, 31, 44, 44, 40, 1, 38, 48, 1, 35, 44, 47, 1, 49, 37, 34, 1, 50, 48, 34, 1, 44]
    assert dataY[0] == 35
    print("Testing successful.")
except:
    print("Tests failed.")

Testing successful.


Running the code to this point shows us that when we split up the dataset into training data for the network to learn that we have just under 160,000 training patterns.

Now that we have prepared our training data we need to transform it so that it is suitable for use with Keras.

First we must transform the list of input sequences into the form [samples, features] expected by an LSTM network (if we would have more features per time step the dimension would be [samples, time_steps, features]).

Finally, we need to convert the output patterns (single characters converted to integers) into a one hot encoding. This is so that we can configure the network to predict the probability of each of the 61 different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character. Each y value is converted into a sparse vector with a length of 61, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents.

In [43]:
# reshape X to be [samples, time steps]
X = numpy.reshape(dataX, (n_patterns, seq_length))
# one hot encode the output variable
Y_onehot = np.eye(len(chars))[dataY]

Now we can define our model. It consists of an embedding layer that embeds each character in a 32-dimensional feature space. The next layer is a layer with 256 LSTM units followed by a Dropout-Layer to reduce overfitting. The last layer is a Dense-Layer used to predict the probabilities for each of the 61 characters.

In [44]:

embedding_vector_length = 32
top_words = n_vocab
model = LSTM_model(num_embeddings = 61,
                 embedding_dim = 32,
                lstm_dim = 256,
                target_size = 61,
                recurrent_dropout=0.15,
                seq_dropout=0.1)
model.to(device)

LSTM_model(
  (embedding): Embedding(61, 32)
  (lstm): LSTMCell(32, 256)
  (clf): Linear(in_features=256, out_features=61, bias=True)
  (recurrent_do): RecurrentDropout(
    (id): Identity()
  )
  (seq_dropout): Dropout1d(p=0.1, inplace=False)
)

We can now fit our model to the data. Here we use a modest number of 20 epochs and a large batch size of 128 patterns.

In [45]:
def train_batch(model, optimizer, loss_fn, dataloader):
    losses = []
    hit = 0
    miss = 0
    model.train()
    for batch in dataloader:
        batch[0] = batch[0].to(device)
        batch[1] = batch[1].to(device)
        output = model(batch[0])
        loss = loss_fn(output.squeeze(), batch[1].squeeze().float())

        losses += [loss.item()]
        loss.backward()
        pred =  output.argmax(axis=1)
        equal = pred.squeeze() == batch[1].argmax(axis=1).squeeze()
        hit += equal.sum()
        miss += (~equal).sum()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()
        optimizer.zero_grad()
    return np.mean(losses), hit/(hit + miss)
def test_batch(model, optimizer, loss_fn, dataloader):
    losses = []
    hit = 0
    miss = 0
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch[0] = batch[0].to(device)
            batch[1] = batch[1].to(device)
            output = model(batch[0])
            loss = loss_fn(output.squeeze(), batch[1].argmax(axis=1).squeeze().float())
            losses += [loss.item()]
            pred =  output.argmax(axis=1)
            equal = pred.squeeze() == batch[1].squeeze()
            hit += equal.sum()
            miss += (~equal).sum()
    return np.mean(losses), hit/(hit + miss)

In [46]:
data = torch.utils.data.TensorDataset(torch.Tensor(dataX).long(),
                                        torch.Tensor(Y_onehot).long())
dataloader = torch.utils.data.DataLoader(data, batch_size = 256, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), 0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
loss_fn = nn.CrossEntropyLoss()

Training this model really takes some time. So you can skip the learning step and use the pretrained model instead.

In [47]:
for epoch in range(20):
    train_loss = train_batch(model, optimizer, loss_fn, dataloader)
    print(f"Epoch: {epoch}, train_loss: {train_loss[0]}, train_acc: {train_loss[1]}")
    scheduler.step(train_loss[0])

KeyboardInterrupt: 

<b>Exercise 5</b>:
    
Load the weights stored in the file  "best_weights.pt" (model with the best weights).

Hint: The file contains a PyTorch's state_dict.

In [50]:
path: str = r'D:\Documents\GitHub\UNI-Machine-Learning-II\Lab 3\best_weights.pt'
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

<All keys matched successfully>

<b>Exercise 6:</b>
    
Also, when preparing the mapping of unique characters to integers, we must also create a reverse mapping that we can use to convert the integers back to characters so that we can understand the predictions. Create a mapping from the integers to the characters as we did by defining the dictionary char_to_int.

In [51]:
int_to_char = {item: key for key, item in char_to_int.items()} # MODIFY

<b>Exercise 7:</b>

Finally, we need to actually make predictions.

The forward method of LSTM_model, has an autoregressive_iterations parameters. Use it to generate text with the LSTM.

We can pick a random input pattern as our seed sequence, then print generated characters as we generate them.

Use the pretrained (or your own trained) LSTM and predict the next 100 characters using a random sequence as starting point. How could you easily obtain different predictions for the next character? (hint torch.multinomial + torch.softmax)

In [57]:
class LSTM_model(nn.Module):
    def __init__(self,
                 num_embeddings = 5000,
                 embedding_dim = 32,
                lstm_dim = 10,
                target_size = 1,
                recurrent_dropout = 0.2,
                seq_dropout = 0.1):
        super().__init__()
        self.lstm_dim = lstm_dim
        self.embedding = nn.Embedding(num_embeddings,
                                     embedding_dim)
        self.lstm = nn.LSTMCell(embedding_dim, lstm_dim)
        self.clf = nn.Linear(lstm_dim, target_size)
        self.recurrent_do = RecurrentDropout(p = recurrent_dropout)
        self.seq_dropout = nn.Dropout1d(p=seq_dropout)

    def forward(self, x, autoregressive_iterations=None, deterministic = True):
        x = self.embedding(x)
        x = self.seq_dropout(x)
        for i in range(x.shape[-2]):
            if i == 0:
                hn, cn = self.lstm(x[:, i])
                hn = self.recurrent_do(hn, first=True)
            else:
                hn, cn = self.lstm(x[:, i], (hn,
                                             cn))
                hn = self.recurrent_do(hn, first=False)
        if autoregressive_iterations is None:
            return self.clf(hn)
        else:
            output = []
            for it in range(autoregressive_iterations):
                if not deterministic:
                    token_ps = None
                    token = None
                else:
                    token = self.clf(hn).argmax(1)
                output += token # add token to the output
                embed_token = self.embedding(token).squeeze().unsqueeze(0)
                hn, cn = self.lstm(embed_token, (hn, cn))
        return torch.stack(output)

In [69]:
# Excercise
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print(''.join([int_to_char[value] for value in pattern]))
resulting_strg = ''
pattern_tensor = torch.Tensor(pattern).long().to(device)[None, ]

model.eval()
with torch.no_grad(): # Avoid gradient computation
    output = model.forward(pattern_tensor, autoregressive_iterations=100)

for character in output.tolist():
    resulting_strg += int_to_char[character]
    
print("\nPrediction: " + resulting_strg)
print("\nDone.")


Seed:
they couldn’t get
them out again. that’s all.’

‘thank you,’ said alice, ‘it’s very interesting. i n

Prediction: ever see it was a little birds and the mouse to herself, ‘i don’t know the work of the sea, the mous

Done.


<b>Exercise 8:</b>
    
Use a sentence with at least 100 characters created by your own and look at the next predicted 100 characters. How does the prediction look like?

In [72]:
# YOUR CODE GOES HERE
my_text = 'Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut l'.tlower()
my_text_embedd = [char_to_int[c] for c in my_text]

AttributeError: 'str' object has no attribute 'tlower'